In [1]:
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# URL of the the weather data download page.
url = 'https://dd.weather.gc.ca/climate/observations/hourly/csv/NS/'

# Station ID of the weather station of interest.
stn_id = '8202000'

# Destination directory to save the CSV files
save_dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge'
            r'\RdrsSample\LstmDatasets\NsWeatherData\Daily')

# Send a GET request to the webpage.
response = requests.get(url)

# Parse the webpage with BeautifulSoup.
soup = BeautifulSoup(response.text, 'html.parser')

# Find all 'a' tags (which define hyperlinks) in the HTML
links = soup.find_all('a')

# Define the time period of interest.
start_date = datetime(2011, 1, 1)
end_date = datetime(2021, 12, 31)

# Loop over the links on the webpage.
for link in links:
    # Get the href attribute of the CSV links.
    if link.get('href').endswith('.csv'):
        # Extract the date from the filename. The date is the 5th element when splitting by '_'. 
        filename = link.get('href')
        date_str = filename.split('_')[4]  
        file_date = datetime.strptime(date_str, '%Y')

        # Check if the file is within the date range and has the correct prefix
        if stn_id in filename and start_date <= file_date <= end_date:
            # Construct the full URL to the CSV file.
            csv_url = url + filename

            # Send a GET request to the CSV URL.
            csv_response = requests.get(csv_url)

            # Create the name format for the saved CSV file. It will contain
            # the station ID and the year of the data.
            csv_name = filename.split('_')[3] + '_' + filename.split('_')[4] + '.csv'

            # Create the full path to the file
            file_path = os.path.join(save_dir, csv_name)

            # Write the contents of the response to a file
            with open(file_path, 'wb') as f:
                f.write(csv_response.content)